# Data Audit Report

This is the first part of our Competition 2 where we performed our preprocessing steps on the data. More details can be found in our [README.MD](README.md) file.

### Importing Our Required Packages and things

In [1]:
#importing required libraries and packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backend as bk
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from pandas import ExcelWriter
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

#set plot style to 'ggplot' and don't limit the view of DF when diaplyhign them to screen
plt.style.use('ggplot')
pd.options.display.max_columns = None

### Read the Data In

In [5]:
#importing our data and resetting our index
df = pd.read_excel('Data/Comp2_Raw_Data.xls')
df2 = pd.read_csv('Data/Demos_Data.csv')
df = df.drop(columns = ['ID'])
df.columns = ['Credit_Limit', 'Gender', 'Education', 'Marriage',  'Age', 'Pay_Sept', 'Pay_Aug', 'Pay_Jul', 'Pay_Jun', 'Pay_May', 'Pay_Apr', 
             'Bill_Amt_Sept', 'Bill_Amt_Aug', 'Bill_Amt_Jul', 'Bill_Amt_Jun', 'Bill_Amt_May', 'Bill_Amt_Apr','Pay_Amt_Sept', 'Pay_Amt_Aug',
             'Pay_Amt_Jul', 'Pay_Amt_Jun', 'Pay_Amt_May', 'Pay_Amt_Apr', 'Default']
df2.head(1)

,Unnamed: 0,Credit_Limit,Gender_M,Gender_F,Edu_College,Edu_Other,Married,Married_Other,Age_Range0,Age_Range1,Age_Range2,Age_Range3
0,0,20000,0,1,1,0,1,0,1,0,0,0


In [23]:
df3 = df[['Credit_Limit', 'Gender', 'Education', 'Marriage', 'Age']].copy()

In [6]:
df2 = df2.drop(['Unnamed: 0'], axis =1)

In [7]:
df.head()

,Credit_Limit,Gender,Education,Marriage,Age,Pay_Sept,Pay_Aug,Pay_Jul,Pay_Jun,Pay_May,Pay_Apr,Bill_Amt_Sept,Bill_Amt_Aug,Bill_Amt_Jul,Bill_Amt_Jun,Bill_Amt_May,Bill_Amt_Apr,Pay_Amt_Sept,Pay_Amt_Aug,Pay_Amt_Jul,Pay_Amt_Jun,Pay_Amt_May,Pay_Amt_Apr,Default
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [8]:
#Copy our target variables to their own df
df_target = df[['Default']].copy()
df_target.head(10)
#change the data type to categorical
df_target['Default'] = pd.Categorical(df_target.Default)
#Drop Default from our target variable df
df = df.drop(['Default'], axis=1)

In [9]:
#chacking that our data was transfered properly.
df_target.head(5)

,Default
0,1
1,1
2,0
3,0
4,0


In [11]:
df2.head()

,Credit_Limit,Gender_M,Gender_F,Edu_College,Edu_Other,Married,Married_Other,Age_Range0,Age_Range1,Age_Range2,Age_Range3
0,20000,0,1,1,0,1,0,1,0,0,0
1,120000,0,1,1,0,0,1,1,0,0,0
2,90000,0,1,1,0,0,1,0,1,0,0
3,50000,0,1,1,0,1,0,0,0,1,0
4,50000,1,0,1,0,1,0,0,0,0,1


### Changing Our DataTypes

We want to make our fields from integer to float so we do that here

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(df3,df_target, test_size= 0.20, random_state=2019)
oversample = pd.concat([X_train,y_train],axis=1)
max_size = oversample['Default'].value_counts().max()
lst = [oversample]
    
for class_index, group in oversample.groupby('Default'):
    lst.append(group.sample(max_size-len(group), replace=True))
X_train = pd.concat(lst)
y_train=pd.DataFrame.copy(X_train['Default'])
del X_train['Default']

In [25]:
xgb_model = xgb.XGBClassifier(seed=2019, min_child_weight=1, max_depth=6, learning_rate= 0.2, n_estimators=150, objective='binary:logistic')
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)

/home/jupyter-bwalsh/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(metrics.accuracy_score(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.83      0.60      0.70      4710
          1       0.28      0.57      0.37      1290

avg / total       0.72      0.59      0.63      6000

[[2807 1903]
 [ 556  734]]
0.5901666666666666


In [28]:
feats = xgb_model.feature_importances_ * 100
cols = df3.columns
for x in range(0,len(cols)):
    print(cols[x],'------------', feats[x])

Credit_Limit ------------ 33.52166
Gender ------------ 6.949153
Education ------------ 12.354049
Marriage ------------ 7.8342752
Age ------------ 39.340866


In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
scores = cross_val_score(xgb_model, X_test, y_test.Default.ravel(), cv=kfold)

In [30]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.04)


## Rerun XGBoost with only top 5 important featrues